In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as s
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from Gaussian_Naive_Bayes_API  import gaussian_nb

In [ ]:
train_data=pd.read_csv("aps_failure_training_set.csv",header=None,skiprows=20)

In [ ]:
train_data.head()

In [ ]:
columns=train_data.iloc[0]
columns

In [ ]:
train_data=pd.read_csv("aps_failure_training_set.csv",header=None,skiprows=21,names=columns)
train_data.head()

In [ ]:
train_data['class'].unique()

In [ ]:
train_data=train_data.dropna(axis=1)
train_data.shape

In [ ]:
train_data.replace(to_replace='na',value=np.nan,inplace=True)
train_data.head()

In [ ]:
train_data.dropna(axis=1,inplace=True,thresh=int(0.8*(train_data.shape[0])))


In [ ]:
train_data.shape

In [ ]:
imputer=SimpleImputer()

In [ ]:
train_data_array=imputer.fit_transform(X=train_data.iloc[:,1:])


In [ ]:
train_data_array.shape

In [ ]:
data_columns=train_data.columns
train_data_labels=train_data['class']
train_data_labels

In [ ]:
train_data=pd.DataFrame(data=train_data_array,columns=data_columns[1:])
train_data

In [ ]:
train_data['Class']=train_data_labels

In [ ]:
train_data.shape[1]-1

In [ ]:
for column in train_data.columns[0:train_data.shape[1]-1]:
    train_data[column]=pd.qcut(x=train_data[column],q=10,duplicates='drop').cat.codes

In [ ]:
train_data

In [ ]:
np.eye(10,10)[train_data['aa_000']]

In [ ]:
np_array_list=list()
for columns in train_data.columns[0:train_data.shape[1]-1]:
    np_array_list.append(np.eye(10,10)[train_data[columns]])
    

In [ ]:
data_array=np.concatenate(np_array_list,axis=1)
data_array.shape

In [ ]:
train_data=pd.DataFrame(data=data_array)
train_data['Class']=train_data_labels

In [ ]:
train_data

In [ ]:
rru_object=gaussian_nb(features=train_data.iloc[:,0:train_data.shape[1]-1],labels=train_data['Class'],data_split_ratio=(0.8,0.2,0.0),apply_pca=True,n_components=100)

In [ ]:
rru_object.X_new

In [ ]:
train_data_new=pd.DataFrame(data=rru_object.X_new)
train_data_new['Class']=train_data_labels

In [ ]:
train_data_new

In [ ]:
np.count_nonzero(train_data_new['Class']=='neg')

In [ ]:
np.count_nonzero(train_data_new['Class']=='pos')

In [ ]:
#to balance the training data we will use smote as the neg>>>>>pos

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote_obj=SMOTE(sampling_strategy='minority',k_neighbors=5)

In [ ]:
x_oversampled,y_oversampled=smote_obj.fit_resample(X=rru_object.X_new,y=train_data['Class'])

In [ ]:
train_data_new=pd.DataFrame(data=x_oversampled)
train_data_new['Class']=y_oversampled

In [ ]:
train_data_new

In [ ]:
x_train_data_new_pos=train_data_new[train_data_new['Class']=='pos'].iloc[0:int(0.8*train_data_new.shape[0]//2)]
print(x_train_data_new_pos.shape)
x_train_data_new_neg=train_data_new[train_data_new['Class']=='neg'].iloc[0:int(0.8*train_data_new.shape[0]//2)]
print(x_train_data_new_neg.shape)
x_cv_data_new_pos=train_data_new[train_data_new['Class']=='pos'].iloc[0:int(0.2*train_data_new.shape[0]//2)]
print(x_cv_data_new_pos.shape)
x_cv_data_new_neg=train_data_new[train_data_new['Class']=='neg'].iloc[0:int(0.2*train_data_new.shape[0]//2)]
print(x_cv_data_new_neg.shape)
train_data_concat=pd.concat([x_train_data_new_pos,x_train_data_new_neg])
cv_data_concat=pd.concat([x_cv_data_new_pos,x_cv_data_new_neg])

In [ ]:
print(train_data_concat.shape)
print(cv_data_concat.shape)

In [ ]:
train_data_concat

In [ ]:
miu_hat_pos=np.array(train_data_concat[train_data_concat["Class"]=="pos"].iloc[:,0:100].mean())
sigma_hat_pos=np.array(train_data_concat[train_data_concat["Class"]=="pos"].iloc[:,0:100].cov())
print(np.linalg.det(sigma_hat_pos))

In [ ]:
miu_hat_neg=np.array(train_data_concat[train_data_concat["Class"]=="neg"].iloc[:,0:100].mean())
sigma_hat_neg=np.array(train_data_concat[train_data_concat["Class"]=="neg"].iloc[:,0:100].cov())
print(np.linalg.det(sigma_hat_neg))

In [ ]:
pos_prior=train_data_concat[train_data_concat["Class"]=="pos"].shape[0]/train_data_concat.shape[0]
neg_prior=train_data_concat[train_data_concat["Class"]=="neg"].shape[0]/train_data_concat.shape[0]

In [ ]:
def mocktest(data):
    inputs=np.array(data.iloc[:,0:100])
    posterior_pos=s.multivariate_normal.pdf(inputs,miu_hat_pos,sigma_hat_pos)*pos_prior
    posterior_neg=s.multivariate_normal.pdf(inputs,miu_hat_neg,sigma_hat_neg)*neg_prior
    boolean_mask= posterior_pos> posterior_neg
    predicted_catogory=pd.Series(boolean_mask)
    predicted_catogory.replace(to_replace=[True,False],value=['pos','neg'],inplace=True)
    return np.array(predicted_catogory)

In [ ]:
predicted_results=mocktest(cv_data_concat)
print("predicted_results",predicted_results)
Actual_results=np.array(cv_data_concat['Class'])
print("Actual_results",predicted_results)

In [ ]:
Accuracy=np.count_nonzero(predicted_results==Actual_results)/Actual_results.shape[0]
Accuracy